In [1]:
import json
import torch

from datasets import load_dataset

from ram import inference_ram_openset as inference
from ram import get_transform
from ram.models import ram_plus

c:\Users\pierc\miniconda3\envs\CS263-MLLM\Lib\site-packages\fairscale\experimental\nn\offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
c:\Users\pierc\miniconda3\envs\CS263-MLLM\Lib\site-packages\fairscale\experimental\nn\offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


### Load POPE benchmark dataset

In [3]:
dataset = load_dataset("lmms-lab/HallusionBench", "default")
dataset = dataset['image']
dataset

Dataset({
    features: ['category', 'subcategory', 'visual_input', 'set_id', 'figure_id', 'sample_note', 'question_id', 'question', 'gt_answer_details', 'gt_answer', 'filename', 'image'],
    num_rows: 951
})

In [4]:
def generate_image_tags(dataset):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Device:', device)

    img_size = 384
    model_weights = "pretrained/ram_plus_swin_large_14m.pth"
    output_path = f"hallusionBench_tags.json"
    with open(output_path, 'w') as f:
        f.write('')

    transform = get_transform(image_size=img_size)

    model = ram_plus(pretrained=model_weights,
                    image_size=img_size,
                    vit='swin_l'
            )
    
    model.eval()
    model = model.to(device)

    prev_img_source = None
    for idx in range(len(dataset)):
        img = dataset['image'][idx]
        img_source = dataset['filename'][idx]

        # Skip duplicate images
        if img_source == prev_img_source:
            continue
        prev_img_source = img_source

        img = transform(img).unsqueeze(0).to(device)

        tags = inference(img, model)
        
        with open(output_path, 'a') as f:
            json.dump({img_source: tags}, f)
            f.write(',\n')

        print(img_source, ": ", tags)

    print('Tags generated and saved to', output_path)


In [5]:
generate_image_tags(dataset)

Device: cuda


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


--------------
pretrained/ram_plus_swin_large_14m.pth
--------------
load checkpoint from pretrained/ram_plus_swin_large_14m.pth
vit: swin_l
./VS/chart/0_1.png :  graph | number | individual
./VS/chart/1_1.png :  area | graph | number | line | map | plot | temperature
./VS/chart/2_1.png :  graph | number | individual | plot
./VS/chart/3_1.png :  graph | number | earth | person | world
./VS/chart/3_2.png :  graph | number
./VS/chart/4_1.png :  graph | number
./VS/chart/4_2.png :  graph | number
./VS/chart/5_1.png :  graph | number | individual | line | plot
./VS/chart/5_2.png :  graph | number | individual | line | plot
./VS/chart/6_1.png :  graph | number | line | sea | plot | screenshot | temperature
./VS/chart/6_2.png :  graph | number | line | plot | screenshot | temperature
./VS/chart/7_1.png :  graph | number | line | plot
./VS/chart/7_2.png :  graph | number | line | plot
./VS/chart/8_1.png :  graph | number
./VS/chart/8_2.png :  graph | number
./VS/chart/9_1.png :  graph | numbe